In [1]:
import os
import pandas as pd
import requests
import time
from datetime import datetime

In [ ]:
# AESO Forecasting URL
AESO_URL = "https://www.aeso.ca/grid/grid-planning/forecasting/wind-and-solar-power-forecasting"
Shrt_Wind = "http://ets.aeso.ca/Market/Reports/Manual/Operations/prodweb_reports/wind_solar_forecast/wind_rpt_shortterm.csv"

In [ ]:
# Folder to store forecast data

download_dir = "D:\\Python Projects\\git_projects\\BESS\\data\\aeso_forecasts"
os.makedirs(download_dir, exist_ok=True)

# 10 minutes frequency for next 12 hours
shrt_wind_forecast_file = os.path.join(download_dir, "wind_rpt_shortterm.csv")
shrt_wind_backup_file = os.path.join(download_dir, "wind_rpt_shortterm_backup.csv")
shrt_wind_forecast_url = "http://ets.aeso.ca/Market/Reports/Manual/Operations/prodweb_reports/wind_solar_forecast/wind_rpt_shortterm.csv"

#Hourly frequency for next 7 days
longt_wind_forecast_file = os.path.join(download_dir, "wind_rpt_longterm.csv")
longt_wind_backup_file = os.path.join(download_dir, "wind_rpt_longterm_backup.csv")
longt_wind_forecast_url = "http://ets.aeso.ca/Market/Reports/Manual/Operations/prodweb_reports/wind_solar_forecast/wind_rpt_longterm.csv"

# 10 minutes frequency for next 12 hours
shrt_solar_forecast_file = os.path.join(download_dir, "solar_rpt_shortterm.csv")
shrt_solar_backup_file = os.path.join(download_dir, "solar_rpt_shortterm_backup.csv")
shrt_solar_forecast_url = "http://ets.aeso.ca/Market/Reports/Manual/Operations/prodweb_reports/wind_solar_forecast/solar_rpt_shortterm.csv"

#Hourly frequency for next 7 days
longt_solar_forecast_file = os.path.join(download_dir, "solar_rpt_longterm.csv")
longt_solar_backup_file = os.path.join(download_dir, "solar_rpt_longterm_backup.csv")
longt_solar_forecast_url = "http://ets.aeso.ca/Market/Reports/Manual/Operations/prodweb_reports/wind_solar_forecast/solar_rpt_longterm.csv"

In [22]:
#To download the short-term wind forecast csv file
def download_csv(url, file_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"Downloaded new forecast data at {datetime.now()}")
    else:
        print(f"Failed to download data: {response.status_code}")

In [51]:
#To download the latest forecast file, create the back up of forecast, update the forecast with latest data

def update_forecast(forecast_file, backup_file, forecast_url):
    if os.path.exists(forecast_file):
        backup_df = pd.read_csv(forecast_file, index_col="Forecast Transaction Date")
        backup_df.index = pd.to_datetime(backup_df.index)
        backup_df.to_csv(backup_file)
        print("Backup created for existing short-term wind forecast data")
    
        download_csv(forecast_url, forecast_file)

        backup_df = backup_df.dropna(subset=["Actual"])  # Drop rows where 'Actual' column is NaN
        
        new_df = pd.read_csv(forecast_file, index_col="Forecast Transaction Date")

        new_df.index = pd.to_datetime(new_df.index)

        combined_df = pd.concat([backup_df, new_df], axis=0)

        combined_df = combined_df.loc[~combined_df.index.duplicated(keep='first')]

        
        combined_df.to_csv(forecast_file)
        print(f"Updated the short-term wind forecast file at {datetime.now()}")
    else:
        download_csv(forecast_url, forecast_file)


    

In [ ]:
#Short-term forecast frequency is 10 minutes, thus those values need to be averaged on hourly basis.

update_forecast(shrt_wind_forecast_file, shrt_wind_backup_file, shrt_wind_forecast_url)


Downloaded new forecast data at 2025-03-07 00:33:56.017500


In [55]:
update_forecast(longt_wind_forecast_file, longt_wind_backup_file, longt_wind_forecast_url)


Downloaded new forecast data at 2025-03-07 00:33:31.247941


In [ ]:
#Short-term forecast frequency is 10 minutes, thus those values need to be averaged on hourly basis.

update_forecast(shrt_solar_forecast_file, shrt_solar_backup_file, shrt_solar_forecast_url)


Downloaded new forecast data at 2025-03-07 00:33:58.594243


In [58]:
update_forecast(longt_solar_forecast_file, longt_solar_backup_file, longt_solar_forecast_url)

Downloaded new forecast data at 2025-03-07 00:34:01.032691


We need to call these functions every hour in Wind_Solar Forecast Inferencing sript